<a href="https://colab.research.google.com/github/cocoisland/DS-Unit-2-Sprint-2-Linear-Regression/blob/master/module3-gradient-descent/Gradient_Descent_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gradient Descent Implementation Challenge!!

## Use gradient descent to find the optimal parameters of a **multiple** regression model. (We only showed an implementation for a bivariate model during lecture.)

A note: Implementing gradient descent in any context is not trivial, particularly the step where we calculate the gradient will change based on the number of parameters that we're trying to optimize for. You will need to research what the gradient of a multiple regression model looks like. This challenge is pretty open-ended but I hope it will be thrilling. Please work together, help each other, share resources and generally expand your understanding of gradient descent as you try and achieve this implementation. 

## Suggestions:

Start off with a model that has just two $X$ variables You can use any datasets that have at least two x variables. Potential candidates might be the blood pressure dataset that we used during lecture on Monday: [HERE](https://college.cengage.com/mathematics/brase/understandable_statistics/7e/students/datasets/mlr/excel/mlr02.xls) or any of the housing datasets. You would just need to select from them the two varaibles $x$ variables and one y variable that you want to work with that you most want to work with. 

Use Sklearn to find the optimal parameters of your model first. (like we did during the lecture.) So that you can compare the parameter estimates of your gradient-descent linear regression to the estimates of OLS linear regression. If implemented correctly they should be nearly identical.

Becoming a Data Scientist is all about striking out into the unknown, getting stuck and then researching and fighting and learning until you get yourself unstuck. Work together! And fight to take your own learning-rate fueled step towards your own optimal understanding of gradient descent! 


In [4]:
##### Make it Hap'n Cap'n #
from google.colab import files
uploaded = files.upload()


Saving mlr02.xls to mlr02.xls


In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io

x=uploaded['mlr02.xls']
s=str(x)

data = io.StringIO(s) 

df=pd.read_excel(x)

ValueError: ignored

In [25]:
df.head()


,b'\t\x00\x04\x00\x00\x00\x10\x00\x00\x00\x08\x00\x00\x00\x0c\x00\x00\x00\x03\x00\x1e\x00\x08\x00\x07General\x1e\x00\x0e\x00\rh:mm:ss AM/PM\x1e\x00\x07\x00\x06m/d/yy\x1e\x00\r\x00\x0cm/d/yy\\ h:mm\x04\x00\n\x00\x00\x00\x00\x00\x00\x00\x00\x02X1\x04\x00\n\x00\x00\x00\x01\x00\x00\x00\x00\x02X2\x04\x00\n\x00\x00\x00\x02\x00\x00\x00\x00\x02X3\x03\x00\x0f\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80`@\x03\x00\x0f\x00\x01\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00J@\x03\x00\x0f\x00\x01\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa0e@\x03\x00\x0f\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xe0a@\x03\x00\x0f\x00\x02\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80M@\x03\x00\x0f\x00\x02\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00g@\x03\x00\x0f\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 c@\x03\x00\x0f\x00\x03\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc0P@\x03\x00\x0f\x00\x03\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00@h@\x03\x00\x0f\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@d@\x03\x00\x0f\x00\x04\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00@R@\x03\x00\x0f\x00\x04\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00`j@\x03\x00\x0f\x00\x05\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@c@\x03\x00\x0f\x00\x05\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00P@\x03\x00\x0f\x00\x05\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80h@\x03\x00\x0f\x00\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00e@\x03\x00\x0f\x00\x06\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80R@\x03\x00\x0f\x00\x06\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80k@\x03\x00\x0f\x00\x07\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 a@\x03\x00\x0f\x00\x07\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00K@\x03\x00\x0f\x00\x07\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80g@\x03\x00\x0f\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa0b@\x03\x00\x0f\x00\x08\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80N@\x03\x00\x0f\x00\x08\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80g@\x03\x00\x0f\x00\t\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xe0c@\x03\x00\x0f\x00\t\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00@P@\x03\x00\x0f\x00\t\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\xe0i@\x03\x00\x0f\x00\n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00`@\x03\x00\x0f\x00\n\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00G@\x03\x00\x0f\x00\n\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\xe0d@\x03\x00\x0f\x00\x0b\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc0d@\x03\x00\x0f\x00\x0b\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00R@\x03\x00\x0f\x00\x0b\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00 k@\n\x00\x00\x00'


## Stretch Goals

If you happen upon the most useful resources for accomplishing this challenge first, I want you to spend time today studying other variations of Gradient Descent-Based Optimizers.

- Try and write a function that can perform gradient descent for arbitarily large (in dimensionality) multiple regression models. 
- Create a notebook for yourself exploring these topics
- How do they differ from the "vanilla" gradient descent we explored today
- How do these different gradient descent-based optimizers seek to overcome the challenge of finding the global minimum among various local minima?
- Write a blog post that reteaches what you have learned about these other gradient descent-based optimizers.

[Overview of GD-based optimizers](http://ruder.io/optimizing-gradient-descent/)

[Siraj Raval - Evolution of Gradient Descent-Based Optimizers](https://youtu.be/nhqo0u1a6fw)